In [2]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
# Carga los archivos en un DataFrame
steam_games = pd.read_csv('../data/steam_games.csv')
users_items = pd.read_csv('../data/users_items.csv')
users_reviews = pd.read_csv('../data/users_reviews.csv')

In [4]:
muestra_steam_games_modelo = steam_games.head(20000)
muestra_users_reviews_modelo = users_reviews.head(20000)



In [18]:
def recomendacion_juego(game_id: int, top_n=5):
    # Crear el vectorizador TF-IDF
    tfidf_vectorizer = TfidfVectorizer()

    # Transformar los datos de géneros en vectores numéricos y entrenar
    tfidf_matrix = tfidf_vectorizer.fit_transform(muestra_steam_games_modelo['genres'])

    # Calcular similitud de coseno entre los vectores
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

    # Buscar el índice del juego con la ID proporcionada
    idx = muestra_steam_games_modelo.index[muestra_steam_games_modelo['item_id'] == game_id].tolist()
    recommended_indices = []  # Inicializar como una lista vacía

    # Continuar con el procesamiento si se encontró alguna coincidencia
    if idx:
        idx = idx[0]
        # Manejar adecuadamente si no se encontró ninguna coincidencia
        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        top_n = min(int(top_n), len(sim_scores))
        recommended_indices = [i[0] for i in sim_scores[1:top_n+1]]

    # Devolver los títulos de los juegos recomendados
    return list(muestra_steam_games_modelo['title'].iloc[recommended_indices])



In [19]:
muestra_steam_games_modelo.columns

Index(['genres', 'title', 'release_date', 'specs', 'price', 'item_id',
       'developer'],
      dtype='object')

In [41]:
def recomendacion_usuario(user_id: str, top_n=5):
    # Filtrar las reseñas del usuario
    user_reviews = muestra_users_reviews_modelo[muestra_users_reviews_modelo['user_id'] == user_id]

    if not user_reviews.empty:
        # Obtener el primer juego de las reseñas del usuario
        game_id = user_reviews['item_id'].iloc[0]

        # Llamar a la función recomendacion_juego con el ID del juego
        recommended_games = recomendacion_juego(game_id=int(game_id), top_n=top_n)

        return recommended_games
    else:
        return 'VACIO'





In [42]:
muestra_users_reviews_modelo.columns

Index(['user_id', 'item_id', 'recommend', 'sentiment_analysis', 'posted_year'], dtype='object')

In [47]:
recomendacion_usuario('doctr')


['Battle Royale Trainer',
 'Battle Royale Trainer',
 'Army of Tentacles: (Not) A Cthulhu Dating Sim: Black GOAT of the Woods Edition',
 'Army of Tentacles: (Not) A Cthulhu Dating Sim: Black GOAT of the Woods Edition',
 'The first thrust of God']